<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/joint_probability_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JOINT PROBABILITY NETWORK
- using just one folder structure (hierarchical for an umbrella network), the program also makes a general softmax model from the bottom level folders, then combines the predictions of the general model with the predictions of the top level umbrella model
- their predictions are combined with joint probability multiplication

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-15 05:05:58--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48219 (47K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  47.09K  --.-KB/s    in 0.001s  

2022-04-15 05:05:58 (34.0 MB/s) - ‘umbrella.py’ saved [48219/48219]



In [3]:
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [4]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

In [5]:
network_type = 'softmax'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 30
levels = 1 # not used, set automatically to 1 and 0
channels = 3
valid_ratio = 0.1 # validation set
test_ratio = 0.0 # test set
initialize_now = True
Model = Joint_Probability_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)

In [6]:
Model.traverse_training_data()

specialist
small_hierarchy trainX 154 (154,) trainY 154 (154, 3)
   animals trainX 0 (0,) trainY 0 (0,)
      alligator trainX 0 (0,) trainY 0 (0,)
      mammals trainX 0 (0,) trainY 0 (0,)
      salamanders trainX 0 (0,) trainY 0 (0,)
      snake trainX 0 (0,) trainY 0 (0,)
   objects trainX 0 (0,) trainY 0 (0,)
      automobiles trainX 0 (0,) trainY 0 (0,)
      bicycles trainX 0 (0,) trainY 0 (0,)
      boats trainX 0 (0,) trainY 0 (0,)
   plants trainX 0 (0,) trainY 0 (0,)
      cactus trainX 0 (0,) trainY 0 (0,)
      hedges trainX 0 (0,) trainY 0 (0,)
      shrubs trainX 0 (0,) trainY 0 (0,)
generalist
small_hierarchy trainX 154 (154,) trainY 154 (154, 10)
   alligator trainX 0 (0,) trainY 0 (0,)
   mammals trainX 0 (0,) trainY 0 (0,)
   salamanders trainX 0 (0,) trainY 0 (0,)
   snake trainX 0 (0,) trainY 0 (0,)
   automobiles trainX 0 (0,) trainY 0 (0,)
   bicycles trainX 0 (0,) trainY 0 (0,)
   boats trainX 0 (0,) trainY 0 (0,)
   cactus trainX 0 (0,) trainY 0 (0,)
   hedges t

In [7]:
Model.make_models()

making models for specialist
making model for small_hierarchy
making modes for generalist
making model for small_hierarchy


In [8]:
Model.train_models(eps=1,batch=10,aug=False,vrb=True)

training specialist
training model for small_hierarchy
trainX 154
trainY 154
16/16 [==============================] - 1s 10ms/step - loss: 1.0875 - accuracy: 0.4610
training generalist
training model for small_hierarchy
trainX 154
trainY 154
16/16 [==============================] - 1s 10ms/step - loss: 2.2986 - accuracy: 0.1234


In [9]:
Model.predict_validation_set()

validation accuracy
specialist
expected [[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]
predicted [[0.23901524799532758, 0.20466032084502983, 0.5563244311596425], [0.30466893621072566, 0.18700332461819819, 0.5083277391710761], [0.24775928403756708, 0.2023086873633195, 0.5499320285991134], [0.33204275566970687, 0.17964137070457417, 0.48831587362571904], [0.22879535671000353, 0.20740887293355173, 0.5637957703564447], [0.3010459398381826, 0.1879776984122483, 0.5109763617495692], [0.19235013929029246, 0.5228618883343682, 0.2847879723753393], [0.15779314407147094, 0.42892623618489956, 0.41328061974362956], [0.1809752354386055, 0.32708307066753584, 0.4919416938938587], [0.19046414178979132, 0.29180064260997207, 0.5177352156002367], [0.19336469213279606, 0.28101557897704665, 0.5256197288901572], [0.27910390917307454, 0.1938788193270664, 0.5270172714998589